# Lec 11: Ridge Regression
## CMSE 381 - Spring 2024



In this module we are going to test out the ridge regression method we discussed in class from Chapter 6.2.

In [ ]:
# Everyone's favorite standard imports
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import time
import seaborn as sns


from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error


# Loading in the data

Ok, here we go, let's play with a baseball data set. 

In [ ]:
hitters_df = pd.read_csv('../../DataSets/Hitters.csv')
hitters_df.head()

Annoyingly enough we have some missing values in the data. 

In [ ]:
print("Number of null values:", hitters_df["Salary"].isnull().sum())


So let's go clean those up.....

In [ ]:
# Print the dimensions of the original Hitters data (322 rows x 20 columns)
print("Dimensions of original data:", hitters_df.shape)

# Drop any rows the contain missing values, along with the player names
hitters_df = hitters_df.dropna().drop('Player', axis=1)

# Print the dimensions of the modified Hitters data (263 rows x 20 columns)
print("Dimensions of modified data:", hitters_df.shape)

# One last check: should return 0
print("Number of null values:", hitters_df["Salary"].isnull().sum())

hitters_df.head()

And finally, we can replace our categorical variables with dummy variables.

In [ ]:
hitters_df = pd.get_dummies(hitters_df, drop_first = True)
hitters_df.head()

In [ ]:
y = hitters_df.Salary

# Drop the column with the independent variable (Salary)
X = hitters_df.drop(['Salary'], axis = 1).astype('float64')

X.info()

## Normalizing the data 

Our first job is to normalize the data. Right now, all our columns have different standard deviations.

In [ ]:
np.std(X)

We'll do it on all the columns, but I'll draw pictures just with the `Hits` column for ease of visualization. Here's that the distribution of that particular column is before doing any normalization.

In [ ]:
sns.histplot(X.Hits)

I'm going to take all my columns and normalize them so that each column has standard deviation 1.  

In [ ]:
def normalize(X):    
  return (X-X.mean())/X.std()
X_norm = normalize(X)
X_norm.std()

Woohoo, I have normalized data! 

Notice that the shape of the `Hits` histogram is the same, we've just scaled the $x$-axis. 

In [ ]:
sns.histplot(X_norm.Hits)

# Ridge Regression

In class, we learned that doing ridge regression means that we try to find the best model accoding to the score
$$
RSS + \lambda \sum_{i} \beta_i^2.
$$
The good news is that `scikitlearn` has a built in `Ridge` function.  

- [Documentation](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.Ridge.html)
- [User guide](https://scikit-learn.org/stable/modules/linear_model.html#ridge-regression)

In [ ]:
from sklearn.linear_model import Ridge

The bad (ok, not honestly that bad) news is that they call their $\lambda$ parameter $\alpha$. So we're just going to minimize 
$$
RSS + \alpha \sum_{i} \beta_i^2.
$$
instead. So if I pick an $\alpha$ value, I can do ridge regression as follows.

In [ ]:
a = 1 #<------ this is me picking an alpha value
X_norm = normalize(X)
# Fit the regression
ridge = Ridge(alpha = a) 
ridge.fit(X_norm, y)

# Get all the coefficients
print('intercept:', ridge.intercept_)
print('\n')
print(pd.Series(ridge.coef_, index = X_norm.columns))
print('\nTraining MSE:',mean_squared_error(y,ridge.predict(X_norm)))

Of course, that was just me picking a random $\alpha$ out of a hat so there's no reason to trust that it's a good one. I could sit here all day and move that $\alpha$ around to see what's going on, but why do that, when I can make a for loop!

Here's a pile of $\alpha$s for us to test on.

In [ ]:
alphas = 10**np.linspace(4,-2,100)*0.5
alphas = np.append(alphas,0)
alphas

First off, let's take a look at how the coefficients learned change for various choices of $\alpha$. 

Associated with each alpha value is a vector of ridge regression coefficients, which we'll store in a matrix coefs. In this case, it is a  19×100  matrix, with 19 rows (one for each predictor) and 100 columns (one for each value of alpha). 



In [ ]:
ridge = Ridge()
coefs = []

for a in alphas:
    model =  Ridge(alpha = a)
    model.fit(X_norm, y)
    coefs.append(model.coef_)
    

coefs = pd.DataFrame(coefs,columns = X_norm.columns)
coefs.head()
   

Let's say I just want to look at the coefficient for the `Hits` column. 


In [ ]:
plt.plot(alphas,coefs.Hits, label = 'Hits')
plt.xscale('log')

plt.legend()

Or just for the `Runs` column.

In [ ]:
plt.plot(alphas,coefs.Runs, label = 'Runs')
plt.xscale('log')

plt.legend()

But that's pretty annoying, so instead we look at all of the coefficients at once. 

In [ ]:
for var in coefs.columns:
    plt.plot(alphas, coefs[var], label = var)
plt.xscale('log')
plt.axis('tight')
plt.xlabel('alpha')
plt.ylabel('coefficients')

plt.legend(bbox_to_anchor = (1.1,1.1))

In [ ]:
# Fix for plotting errors 
# also added grey color for ease of visualization
for var in coefs.columns:
    if np.abs(coefs[var][100])<200:
        plt.plot(alphas, coefs[var], label = var, color = 'grey', linewidth = .5)
    else:
        plt.plot(alphas, coefs[var], label = var)
    
plt.xscale('log')
plt.axis('tight')
plt.xlabel('alpha')
plt.ylabel('coefficients')

plt.legend(bbox_to_anchor = (1.1,1.1))

## Train/test split version


Now we can start setting up the usual train/test splits to have at least a starting idea of how the testing error is going. The `random_state=1` bit just makes it so that everyone should get the same random split. 

In [ ]:
# Split data into training and test sets
X_train, X_test , y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=1)


&#9989; **<font color=red>Do this:</font>** Train a model using ridge regression with $\alpha = 4$. What is the MSE of your model on the testing set?

In [ ]:
# Your code here #


&#9989; **<font color=red>Do this:</font>** Ha ha nah, you can do better than that.  Lets try all our alphas and take a look at the testing MSE to make a better decision about what $\alpha$ we might want. Modify the code below to plot your testing MSE for all the alphas. What $\alpha$ should we use to train the model?

In [ ]:
# Modify your code from above and add it in the for loop to plot the testing MSE

ridge = Ridge()
errors = []

for a in alphas:
    # ==== Your code goes in here ==== #
    errors.append(17) #<----- random number in here so that the code runs before you fix it

#---plotting----

plt.plot(alphas,errors)
plt.title('Testing MSE')
ax=plt.gca()
ax.set_xscale('log')
ax.set_yscale('log')



## RidgeCV

Whelp, your meanie professor didn't tell you that there's actually a built in function to do this for you (sorry-not-sorry). Aren't you glad you didn't read ahead?


In [ ]:
from sklearn.linear_model import RidgeCV

- [Documentation](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.RidgeCV.html)
- [User Guide](https://scikit-learn.org/stable/modules/linear_model.html#ridge-regression)

Basically, `RidgeCV` runs LOOCV (unless you tell it otherwise, see the [documentation](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.RidgeCV.html)) on all the $\alpha$ values you specify on an input array, and tells you the best $\alpha$ given that. 

First, I am going to normalize. I'm being careful to not just normalize all of $X$ and then pass this matrix on since this is an example of [data leakage](https://scikit-learn.org/stable/common_pitfalls.html#data-leakage-during-pre-processing). In that version, we are using some amount of information about the testing data (in this case it is contributing to the normalization process) to affect something in our training process. So here's what we do instead. 

In [ ]:
X_train, X_test , y_train, y_test = train_test_split(X_norm, y, test_size=0.3, random_state=1)



In [ ]:
# I'm going to drop that 0 from the alphas because it makes 
# RidgeCV cranky
alphas = alphas[:-1]


ridgecv = RidgeCV(alphas = alphas, 
                  scoring = 'neg_mean_squared_error', 
                  )
ridgecv.fit(X_train, y_train)



print('alpha chosen is', ridgecv.alpha_)

I can predict my values on the test set directly from the `ridgecv` model we just built. 

In [ ]:
pred = ridgecv.predict(X_test)
mean_squared_error(y_test,pred)

This is exactly the same result as if I went and retrained my model using the chosen $\alpha$ using `Ridge`. 

In [ ]:
ridge = Ridge(alpha = ridgecv.alpha_)
ridge.fit(X_train, y_train)
mean_squared_error(y_test, ridge.predict(X_test))

&#9989; **<font color=red>Do this:</font>** Why did we get a different best choice of $\alpha$ than we found in the previous section? 

*Your answer here*